In [ ]:
import numpy as np
import pandas as pd
import os
import time
import string
from tqdm import tqdm_notebook as tqdm


### define input and output directory

In [ ]:
patients_f = '[the output from pretraining_labeling.py]'
OUTPATH = '[output directory]'

In [ ]:
import pickle

with open(patients_f, 'rb') as f:
    patients = pickle.load(f)

In [ ]:
len(patients)

In [ ]:
psk_hospit_f = '[the file containg the mapping of patient sk and the hispotal ID]'
psk_hospit_df = pd.read_csv(psk_hospit_f, sep='\t')

In [ ]:
psk_hospit_dict = {}
for idx, row in psk_hospit_df.iterrows():
    psk_hospit_dict[row['patient_sk']] = row['hospital_id']

In [ ]:
p_patients_factors = {}
n_patients_factors = {}

for d in patients:
    factors = {}
    factors['yob'] = d['yob']
    factors['gender'] = d['gender']
    factors['race'] = d['race']
    factors['lung_dt'] = d['lung_dt']
    factors['first_dt'] = d['first_dt']
    factors['final_dt'] = d['final_dt']
    factors['hospitalid'] = psk_hospit_dict[d['patient_sk']]
    if d['label'] == 0:
        n_patients_factors[d['patient_sk']] = factors
    elif d['label'] == 1:
        factors['bm_dt'] = d['bm_dt']
        p_patients_factors[d['patient_sk']] = factors

In [ ]:
len(p_patients_factors)

In [ ]:
match_dict = {}
for k in tqdm(list(p_patients_factors.keys())):
    #print(k)
    v = p_patients_factors[k]
    #print(v)
    if len(n_patients_factors) == 0:
        print('Candidate control list null!')
        break
    matched_psk = []
    for k1, v1 in n_patients_factors.items():
        ### hospital match
        if v['hospitalid'] != v1['hospitalid']:
            continue
        ### gender match
        if v['gender'] != v1['gender']:
            continue
        ### race match
        if v['race'] != v1['race']:
            continue
        ### yob match
        if abs(v['yob'] - v1['yob']) > 5:
            continue
        ### lung_dt match
        if abs((v['lung_dt'] - v1['lung_dt']).days) > 180:
            continue
        ### final visit dt match
        if (v['bm_dt'] - v1['final_dt']).days > 30:
            continue
        ### first visit dt match
        if v1['lung_dt']  == v1['first_dt']:
            continue
        ### have visits before lung_dt
        if abs((v['first_dt'] - v1['first_dt']).days) > 365:
            continue
        
        matched_psk.append(k1)
        if len(matched_psk) >= 10:
            break
    for tmp_sk in matched_psk:
        n_patients_factors.pop(tmp_sk)
    #print(f'{len(matched_psk)} control(s) matched for case {k}')
    if len(matched_psk) > 0:
        match_dict[k] = matched_psk

In [ ]:
matched_ct = 0
matched_n_ct = 0
for k,v in match_dict.items():
    n_matched = len(v)
    if n_matched > 0:
        matched_ct += 1
        matched_n_ct += n_matched
print(matched_ct)
print(len(p_patients_factors))
print(matched_n_ct)
print(matched_n_ct / matched_ct)

In [ ]:
print('Saving data by pickle')
with open(os.path.join(OUTPATH, 'matching_dict_same_hospital.pickle'), 'wb') as f:
    pickle.dump(match_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
    